In [28]:
import numpy as np
import torch
import torch.nn as nn      
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import cv2
from PIL import Image
from pylab import rcParams
rcParams['figure.figsize'] = 5, 5
import tqdm
from numpy import moveaxis
import scipy.ndimage
from IPython.display import clear_output
import time
import pyautogui
from Xlib import display
from mss.linux import MSS as mss
import mss.tools

animals = ["chicken", "pig", "panda", "terrain"]

In [2]:
def screen_cap(top = 0, left = 0, width = 650, height = 480):
    mon = {"top": top, "left": left, "width": width, "height": height}
    sct = mss.mss()
    img = np.array(sct.grab(mon))[...,:3]
    
    sct.close()
    return img

In [4]:
def shoot_arrow():
    pyautogui.click(button="right")
    
def reload_arrow(time_taken):
    pyautogui.mouseDown(button = "right")
    time.sleep(time_taken)
    pyautogui.mouseUp(button = "right")

    
def strike_with_sword():
    pyautogui.click(button="left")


def normalise_targets(target, actual_image_size, small_image_size):
    
    new_x = int((target[0]/small_image_size[0])*actual_image_size[0])
    new_y = int((target[1]/small_image_size[1])*actual_image_size[1])    
    return [new_x, new_y]
    

def move_mouse_co_ord_x(x_val , time = 0.001):
    y_val = find_mouse(y = True)
    pyautogui.moveTo(x_val,y_val, duration= time )
    
def move_mouse_co_ord(x_val, y_val , time = 0.001):
    pyautogui.moveTo(x_val,y_val, duration= time )
    
    
def find_mouse(x = False , y = False):
    
    data = display.Display().screen().root.query_pointer()._data
    res = [data["root_x"], data["root_y"]]
    if x == True and y == True:
        return res
    elif x == True and y == False:
        return res[0]
    else:
        return res[1]

In [80]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 7, 5)
        self.pool = nn.MaxPool2d(3, 3)
        
        self.dropout = nn.Dropout(p=0.5)
        self.conv2 = nn.Conv2d(7, 10, 5)
        
        self.fc1 = nn.Linear(10, 10)
        self.fc2 = nn.Linear(10, 8)
        self.fc3 = nn.Linear(8, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
hunter = Net()
hunter.load_state_dict(torch.load("hunter.pt"))
hunter.eval()

Net(
  (conv1): Conv2d(3, 7, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (conv2): Conv2d(7, 10, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=10, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
)

In [81]:
def fgsm(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
        # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
        
    perturbed_image = torch.clamp(perturbed_image, 0, 255)
    
    return perturbed_image


In [82]:
def negative_epsilon_fgsm_on_tensor(input_tensor, model, epsilon, target_label):
    
    loss = nn.CrossEntropyLoss()
    
    input_tensor.requires_grad = True  # gradients 
    label = torch.tensor([target_label], dtype=torch.long)  
    pred = model(input_tensor)
    loss_val = loss(pred,label)
    loss_val.backward() #backprop
    data_grad = input_tensor.grad.data
    
    bamboozle_tensor = fgsm(input_tensor, epsilon, data_grad)
   
    return bamboozle_tensor

In [107]:
def return_target_numpy(input_np_arr, model, kernel_size, stride, thresh_prob, attack = False, attack_label = 0):
    image = input_np_arr
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    width, height = image.shape[1], image.shape[0]
    circles = []
    
    final_index = -1
    animal_indexes = []
    move = False
    image_ret = image.copy()
    for h in range(0, height):
        for w in range(0, width):
            h_start = h*stride
            w_start = w*stride
            h_end = min(height, h_start + kernel_size)
            w_end = min(width, w_start + kernel_size)
            if (w_end) >= width or (h_end) >= height:
                continue
            np_slice = image[h_start:h_end, w_start:w_end, :]
            if 0 in np_slice.shape:
                continue
            np_slice = cv2.resize(np_slice, (25,25))   
            image_moveaxis =  np.moveaxis(np_slice, 2, 0)    
            sweet_tensor =  torch.from_numpy(image_moveaxis).unsqueeze(0).float()
            
            if attack == True:
                sweet_tensor = negative_epsilon_fgsm_on_tensor(sweet_tensor, model, epsilon = -20, target_label = attack_label)
                
            output_dist = model(sweet_tensor).flatten()
            output_dist_soft = torch.softmax(output_dist, dim = 0)            
            index = torch.argmax(output_dist_soft).item()
            prob = output_dist_soft[index].item()
            
            if index <= 2: ## dont detect terrain
                
                if  prob > thresh_prob:
                    animal_indexes.append(index)
                    circle_center = [int((w_end+w_start)/2), int((h_end+h_start)/2)]
                    circles.append(circle_center)
                else:
                    pass
    try:        
        final_index =  np.bincount(animal_indexes).argmax()

    except:
        pass
    target = np.array([])
    if len(circles) != 0:
        move = True
        target = np.mean(np.array(circles), axis = 0).astype(np.uint8)
    
    return target, move, final_index

In [30]:
def mouse_movement_and_strike(target, top_offset):
        try:
            # move mouse to a certain pos
            move_mouse_co_ord(x_val = target[0] , y_val = target[1] + top_offset, time = 0.0001)
            strike_with_sword()
        except:
            #panic mode basically
            
            cv2.destroyAllWindows() 


In [110]:
font = cv2.FONT_HERSHEY_DUPLEX  
# fontScale 
fontScale = 1
   
# Blue color in BGR 
color = (0, 255, 255) 
  
# Line thickness of 2 px 
thickness = 2


In [114]:
#  some time to breathe and pray that it doesn't embarass me by tracking sunflowers instead of chickens
time.sleep(3)  

attack = False

while True:
    
    top_offset = 150
    
    center = (460, 180 )  # crosshairs on the screen
    start = time.time()
    frame = screen_cap(top= top_offset, height = 550 - top_offset, width = 800)
    
    #resize into a smaller image size 
    smaller_frame = cv2.resize(frame, (int(frame.shape[1]/6), int(frame.shape[0]/6)))
    
    
    target, move, index = return_target_numpy(smaller_frame ,hunter, 20, 12, 0.99, attack = attack, attack_label = 1)
    
    #move mouse only when you see a chicken
    if move == True:
        print("found chicken at = ", target)
        target = normalise_targets(target, frame.shape[:-1], smaller_frame.shape[:-1])
        
        # drawing circles and lines because why not, slower but cooler looking pipeline
        frame = cv2.circle(frame, tuple(target), radius = 50, color = (0, 100, 255) , thickness = 2)
        frame = cv2.line(frame, center, tuple(target), color = (0,0,255), thickness = 5)

        
        frame = cv2.putText(frame, animals[index], tuple(target), font,fontScale, color, thickness, cv2.LINE_AA) 
                            
                            # weird flex by showing the live feed with circles and lines       
    cv2.imshow("hunter_fpv",frame)
    
    #press q to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows() 
        break
      
    # metrics
    time_taken = time.time() - start
    clear_output(wait = True)

    print(int(1/time_taken), " FPS")


11  FPS
found chicken at =  [67 31]


In [ ]:
cv2.destroyAllWindows() 